In [5]:
# Boilerplate from Randy cheatsheet at https://docs.google.com/document/d/1utZuLHcKQEZNXTQLOysTNCxTHrqxczAUymmtplpn27Q/edit#
import pandas as pd
import geopandas as gpd
import numpy as np
from geopandas import GeoSeries, GeoDataFrame
%matplotlib inline
pd.set_option('display.max_columns', 500)

In [6]:
# Wide display
from IPython.core.display import display, HTML
display(HTML("<style>#notebook-container { margin-left:-14px; width:calc(100% + 27px) !important; }</style>"))

In [7]:
import csv, json, math, os, numbers, pandas, re, scipy, scipy.sparse, shutil
import struct, subprocess, sys, threading, time, urllib2

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('timelapse-utilities.ipynb')

In [8]:
# Built-in time handling fails for times before 1900.  Use arrow instead.  
# See https://arrow.readthedocs.io/en/latest/ for info
import arrow

def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

# This does the same as above, but takes a GeoJSON point (which is what centroid returns)
def PointToPixelXY(point):
    lon=point.x
    lat=point.y
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

def GetCentroid(geometry):
    s = shape(geometry)
    return (s.centroid.x, s.centroid.y)   

def GetEpoch(date):
    return (date - datetime.datetime(1970, 1, 1)).total_seconds()

def HexToRgb(hex_string):
    rgb = colors.hex2color(hex_string)
    r,g,b = tuple([int(255*x) for x in rgb])
    return (r,g,b)

In [9]:
import json, os, re, sqlite3, thread, urllib, urllib2
from shapely.geometry import Point
import collections


# Google geocoding API to geocode an address
# Assumes key is in google_api_key.txt
# Use as second level lookup only for those which fail census lookup

re_noalnum = re.compile('^([^\w]+)$')

def geocode_address_google(address):
    if(pandas.isnull(address) or re_noalnum.match(address)):
        #print "Skipping address lookup for %r, no alphanumeric characters" %(address)
        return None

    try:
        geocode_address_google.conn
    except:
        geocode_address_google.conn = sqlite3.connect('geocoding_cache')
        geocode_address_google.cur = geocode_address_google.conn.cursor()
        geocode_address_google.cur.execute(('CREATE TABLE IF NOT EXISTS kvs'
                                     ' (key PRIMARY KEY, value)'
                                     ' WITHOUT ROWID;'))
        geocode_address_google.conn.commit()

    geocode_address_google.cur.execute('SELECT value FROM kvs WHERE key=?', (address,))
    rows = geocode_address_google.cur.fetchall()
    if rows:
        #print "Google: Found %r in cache" % (address)
        return json.loads(rows[0][0])
        
    #print "Google: %r not found in cache, fetching" % (address)

    api_key = open('google-api-key-do-not-commit.txt').read()
    payload = {'address':address, 'key':api_key}
    result = json.load(urllib2.urlopen('https://maps.googleapis.com/maps/api/geocode/json?%s' % urllib.urlencode(payload)))
    
    geocode_address_google.cur.execute(('INSERT OR REPLACE INTO kvs (key, value)'
                                 ' VALUES (?, ?);'),
                                 (address, json.dumps(result)))
    geocode_address_google.conn.commit()
    
    # If the result succeeded and generated a formatted address, cache under that key too
    try:
        if(result['status']=='OK'):
            if(len(result['results'])== 1):
                canonical = result['results'][0]['formatted_address']
                #print "Google: Storing canonical addr of %r for %r" % (canonical, address)
                geocode_address_google.cur.execute(('INSERT OR REPLACE INTO kvs (key, value)'
                                                     ' VALUES (?, ?);'),
                                                     (canonical, json.dumps(result)))
                geocode_address_google.conn.commit()
            else:
                print "Google: Fetch %r seemed to succeed, but results empty or non-unique" % (address)

        else:
            print "Google: Fetch %r failed" % (address)

    except:
        #pass
        raise
    
    return result

def get_canonical_address_google(raw_addr):
    if(pandas.isnull(raw_addr) or re_noalnum.match(raw_addr)):
        #print "Skipping address lookup for %r, no alphanumeric characters" %(raw_addr)
        return None

    try:
        result=geocode_address_google(raw_addr)
        if(result['status']=='OK' and len(result['results'])== 1):
            return(result['results'][0]['formatted_address'])
        else:
            return None
    except:
        return None

def get_canonical_zip_google(addr):
    if(pandas.isnull(addr) or re_noalnum.match(addr)):
        #print "Skipping address lookup for %r, no alphanumeric characters" %(addr)
        return None

    try:
        result=geocode_address_google(addr)
        if(result['status']=='OK' and len(result['results'])== 1):
            addr_comps = result['results'][0]['address_components']
            for i in range(0, len(addr_comps)):
                if('postal_code' in addr_comps[i]['types']):
                    return addr_comps[i]['short_name']
        
        return None
    except:
        return None
    
def get_canonical_coords_google(addr):
    if(pandas.isnull(addr) or re_noalnum.match(addr)):
        #print "Skipping address lookup for %r, no alphanumeric characters" %(addr)
        return None

    try:
        result=geocode_address_google(addr)
        if(result['status']=='OK' and len(result['results'])== 1):
            latlon = result['results'][0]['geometry']['location']
            if(latlon):
                return(Point(latlon['lng'], latlon['lat']))
        return None
    except:
        return None


In [60]:
#p1_path = "https://docs-proxy.cmucreatelab.org/spreadsheets/d/1VOcb3tVJgWiFt1xPHqhy_3_8rWI6-YFbyQAnDmnIJpc/export?format=csv&gid=1671023605"
p1_path = "ppt/Survey Part One (Recovered) (Responses) - Survey Part 1.csv"
p1_df = pd.read_csv(p1_path,dtype=object,keep_default_na=False)
p1_df

,Timestamp,Name? Date?,Phone Number?,Email Address?,Where did you live? Specific address and neighborhood?,How much was your rent?,Were utilities included in the rent cost?,Why did you move?,When did you move? Month/Year?,Where was your childcare located?,What means of transportation did you typically use to get to and from your childcare?,"How long was the trip to get to and from your childcare, including walking and wait times for transit?",How much did you pay for transportation to get to and from your childcare?,How frequently did you go to your childcare?,Where was your social/community activities located?,What means of transportation did you typically use to get to and from your social/community activities ?,"How long was the trip to get to and from your social/community activities, including walking and wait times for transit?",How much did you pay for transportation to get to and from your social/community activities?,How frequently did you go to your social/community activities?,Where was your place of worship located?,What means of transportation did you typically use to get to and from your place of worship?,"How long was the trip to get to and from your place of worship, including walking and wait times for transit?",How much did you pay for transportation to get to and from your place of worship?,How frequently did you go to your place of worship?,"Where was your grocery store, farmers market, or food banks/pantry located?","What means of transportation did you typically use to get to and from your grocery store, farmers market, or food banks/pantry?","How long was the trip to get to and from your grocery store, farmers market, or food banks/pantry, including walking and wait times?","How much did you pay for transportation to get to and from your grocery store, farmers market, or food banks/pantry?","How frequently did you go to your grocery store, farmers market, or food banks/pantry?",Where was your doctor's office located?,What means of transportation did you typically use to get to and from your doctor's office?,"How long was the trip to get to and from your doctor's office, including walking and wait times for transit?",How much did you pay for transportation to get to and from your doctor's office?,How frequently did you go to your doctor's office?,Where was your job located?,What means of transportation did you typically use to get to and from your job?,"How long was the trip to get to and from your job, including walking and wait times for transit?",How much did you pay for transportation to get to and from your job?,How frequently did you go to your job?,Age at the time of your move?,Email Address
0,11/3/2017 16:57:26,Lisa Gonzalez,412-478-4459,lisajgonzalez@aol.com,2500 Park Hill Dr. Pgh PA,$500-750,,No affordable housing found with voucher,12/1999,N/A,,,,,Uptown / Northside,"Bus, Friend's car",15-30 min,$48,Twice a month,Beachview,Bus,30-60 min,$48,Once a week,Strip district,Bus,60-90 min,$48,Once a month,"Mercy hospital, uptown",Bus,1 45 mins,$48,Once a month,Board of education/ Northside,Bus,2 1/2,$48,5x a week,,
1,11/6/2017 17:21:34,Nicola Hor'Tonepomper,412-918-9916,Nehortone@gmail.com,,$300-$499,,Closer to work,07/2010,"Mckeesport, PA",Bus,2 hours,Monthly pass,5x a week,Baldwin,Bus,2 hours,Monthly,Every 6 months,Homewood/ Rankin,Bus,2 hours,Weekly/Monthly/Yearly Bus Pass,Once a week,Giant Eagle,Bus,15-30 min,Weekly/Monthly/Yearly Bus Pass,Once a week,East Liberty,Bus,30-60 min,$48,Every 3 months,"McKeesport, PA",Bus,4 hours,Monthly pass/ half date for kids,5x a week,,
2,11/9/2017 14:20:46,Nicole Sled,412-251-1394,nicole.sled@yahoo.com,highland park,$300-$499,No,Landlord sold building without notification,2002,downtown,Bus,15-30 min,Weekly/Monthly/Yearly Bus Pass,5x a week,mom Highland Park,Walk,1-15 min,FREE,Everyday,Hill District,Car,15-30 min,FREE,Once a week,Giant Eagle/ Strip District,"Bus, Car",15-30 min,free parking,2 or 3 times a month,(Hill District) Hill House,Bus,15-30 min,Weekly/Monthly

In [61]:
p1_df = p1_df.rename(columns={'Name?                                                              Date?':'Name',
                              'Where did you live? Specific address and neighborhood?':'start_address',
                              'When did you move? Month/Year?': 'Date'
                             })

In [62]:
p1_df['Name']=p1_df['Name'].apply(lambda x: x.strip())

In [63]:
len(p1_df)

70

In [ ]:
list(p1_df)

In [64]:
#p2_path = "https://docs-proxy.cmucreatelab.org/spreadsheets/d/1VYDs_iQDTpoeLHrv2PBS_UHA8UyKgOsF9vAAe29T-e0/export?format=csv&gid=2870402"
p2_path = "ppt/PPT Housing%2FTransit Survey Part 2 (Responses) - Form Responses 1.csv"
p2_df = pd.read_csv(p2_path,dtype=object,keep_default_na=False)
p2_df

,Timestamp,Name? Date?,How old are you now?,Where do you live? Specific address and neighborhood?,How much is your rent?,Are utilities included in the rent cost?,Where is your childcare located?,What means of transportation do you use to get to and from your childcare?,"How long is the trip to get to and from your childcare, including walking and wait times for transit?",How much do you pay for transportation to get to and from your childcare?,How frequently do you go to your childcare?,Where are your social/community activities located?,What means of transportation do you typically use to get to and from your social/community activities?,"How long is the trip to get to and from your social/community activities, including walking and wait times for transit?",How much do you pay for transportation to get to and from your social/community activities?,How frequently do you go to your social/community activities?,Where is your place of worship located?,What means of transportation do you use to get to and from your place of worship?,"How long is the trip to get to and from your place of worship, including walking and wait times for transit?",How much do you pay for transportation to get to and from your place of worship?,How frequently do you go to your place of worship?,"Where is your grocery store, farmers market, or food banks/pantry located?","What means of transportation do you use to get to and from your grocery store, farmers market, or food banks/pantry?","How long is the trip to get to and from your grocery store, farmers market, or food banks/pantry, including walking and wait times for transit?","How much do you pay for transportation to get to and from your grocery store, farmers market, or food banks/pantry?","How frequently do you go to your grocery store, farmers market, or food banks/pantry?",Where is your doctor's office located?,What means of transportation do you typically use to get to and from your doctor's office?,"How long is the trip to get to and from your doctor's office, including walking and wait times for transit?",How much do you pay for transportation to get to and from your doctor's office?,How frequently do you go to your doctor's office?,Where is your job located?,Did you have to change jobs?,If so have you been able to find employment with similar amount of pay?,What means of transportation do you use to get to and from your job?,"How long is the trip to get to and from your job, including walking and wait times for transit?",How much do you pay for transportation to get to and from your job?,How frequently do you go to your job?,How has the move affected your health and well-being?,What percentage of your monthly income do you spend on housing and transportation costs?,Email Address
0,11/3/2017 17:12:00,Lisa Gonzalez,,2500 brownfield rd pgh pa,$126 voucher,,N/A,,,,,Northside,Bus,2 hours,$97.50,Once a month,N/A,,,,,Wal-Mart century square,Bus,60-90 min,$97.50,Twice a month,Mercy hospital uptown,Bus,2 hours,$97.50,Once a month,SHADYSIDE,,,Bus,2 hours,$97.50,5x a week,Stressful to wonder about if and when I'll be ...,Less than 25%,
1,11/6/2017 17:39:57,Nicola Hor'Tonepomper,,"Mckeesport, Evans Ave",$500-$750,,N/A,,,,,"Baldwin,.PA",Bus,2 hours,Weekly/Monthly/Yearly Bus Pass,Every 6 months,"Braddock, PA",Bus,61-90 mins,Weekly/Monthly/Yearly Bus Pass,Once a week,"Shop n Save, Giant Eagle mckeesport","Bus, Uber/Lyft/Jitney",1-15 min,Weekly/Monthly/Yearly Bus Pass,2 x a month,1811 blvd of the allies Hill district,Bus,30-60 min,$97.50,2-4 times a year,"Northside, post office",,,Bus,60-90 min,Weekly/Monthly/Yearly Bus Pass,5x a week,More sad depressing,More than 75%,
2,11/9/2017 14:38:34,Nicole Sled,,Duqesne,$500-$750,Yes,N/A,,,,,"Mom, Grandfather / Hill District",Bus,3 hours,$9.00 - $11.00,2-4 times a year,N/A,,,,,Strip District,Friend's Car,30-60 min,Free,Once every two weeks,Hill District Broughs Street,Friend's Car,30-60 min,Free,2-4 times a year,N/A,,,,,,,A whole lot. Son's health got bad. Transport.

In [65]:
p2_df = p2_df.rename(columns={'Name?                                                          Date?         ':'Name',
                              'Where do you live? Specific address and neighborhood?':'dest_address'
                             })

In [66]:
p2_df['Name']=p2_df['Name'].apply(lambda x: x.strip())

In [67]:
len(p2_df)

66

In [ ]:
list(p2_df)

In [68]:
ppt_df = pd.merge(p1_df, p2_df, on='Name', copy=True, indicator=False,
                         validate=None, how='outer')

In [69]:
len(ppt_df)

71

In [70]:
ppt_df

,Timestamp_x,Name,Phone Number?,Email Address?,start_address,How much was your rent?,Were utilities included in the rent cost?,Why did you move?,Date,Where was your childcare located?,What means of transportation did you typically use to get to and from your childcare?,"How long was the trip to get to and from your childcare, including walking and wait times for transit?",How much did you pay for transportation to get to and from your childcare?,How frequently did you go to your childcare?,Where was your social/community activities located?,What means of transportation did you typically use to get to and from your social/community activities ?,"How long was the trip to get to and from your social/community activities, including walking and wait times for transit?",How much did you pay for transportation to get to and from your social/community activities?,How frequently did you go to your social/community activities?,Where was your place of worship located?,What means of transportation did you typically use to get to and from your place of worship?,"How long was the trip to get to and from your place of worship, including walking and wait times for transit?",How much did you pay for transportation to get to and from your place of worship?,How frequently did you go to your place of worship?,"Where was your grocery store, farmers market, or food banks/pantry located?","What means of transportation did you typically use to get to and from your grocery store, farmers market, or food banks/pantry?","How long was the trip to get to and from your grocery store, farmers market, or food banks/pantry, including walking and wait times?","How much did you pay for transportation to get to and from your grocery store, farmers market, or food banks/pantry?","How frequently did you go to your grocery store, farmers market, or food banks/pantry?",Where was your doctor's office located?,What means of transportation did you typically use to get to and from your doctor's office?,"How long was the trip to get to and from your doctor's office, including walking and wait times for transit?",How much did you pay for transportation to get to and from your doctor's office?,How frequently did you go to your doctor's office?,Where was your job located?,What means of transportation did you typically use to get to and from your job?,"How long was the trip to get to and from your job, including walking and wait times for transit?",How much did you pay for transportation to get to and from your job?,How frequently did you go to your job?,Age at the time of your move?,Email Address_x,Timestamp_y,How old are you now?,dest_address,How much is your rent?,Are utilities included in the rent cost?,Where is your childcare located?,What means of transportation do you use to get to and from your childcare?,"How long is the trip to get to and from your childcare, including walking and wait times for transit?",How much do you pay for transportation to get to and from your childcare?,How frequently do you go to your childcare?,Where are your social/community activities located?,What means of transportation do you typically use to get to and from your social/community activities?,"How long is the trip to get to and from your social/community activities, including walking and wait times for transit?",How much do you pay for transportation to get to and from your social/community activities?,How frequently do you go to your social/community activities?,Where is your place of worship located?,What means of transportation do you use to get to and from your place of worship?,"How long is the trip to get to and from your place of worship, including walking and wait times for transit?",How much do you pay for transportation to get to and from your place of worship?,How frequently do you go to your place of worship?,"Where is your grocery store, farmers market, or food banks/pantry located?","What means of transportation do you use to get to and from your grocery store, farmers m

In [71]:
ppt_df[['Name','Date','start_address', 'dest_address']]

,Name,Date,start_address,dest_address
0,Lisa Gonzalez,12/1999,2500 Park Hill Dr. Pgh PA,2500 brownfield rd pgh pa
1,Nicola Hor'Tonepomper,07/2010,,"Mckeesport, Evans Ave"
2,Nicole Sled,2002,highland park,Duqesne
3,Ralph Williams,02/2017,307 North Matilda St. Garfield,"Allentown, upper Southside"
4,Ms. Mary Beth,08/2016,,"2100 Duquesne Pl Dr, Duquesne, PA"
5,Heidi Morris,12/2017,7529 Hamilton ave / Homewood,South 21st south 711/ Southside
6,Linda Warman,2014,Hazelwood 203 Glenn Caladh St. pgh pa 15207,"2601 Duquesne Place Drive, Duquesne PA 15110"
7,Renee,Oct 2017,Clairton,Clairton Senior Bldg
8,Quay,Jan 18th 2018,"Jennyland, McKeesport",NaN
9,Candace Lowery,2017,Monroeville,NaN


In [72]:
ppt_oops_df = ppt_df[(ppt_df.start_address.fillna('')== '') | (ppt_df.dest_address.fillna('')=='') | (ppt_df.dest_address.fillna('')=='N/A')]
ppt_oops_df[['Name','Date','start_address', 'dest_address']].sort_values(by=['Name'])


,Name,Date,start_address,dest_address
9,Candace Lowery,2017,Monroeville,NaN
22,Christina Oliver,3 2016,,5001 2nd Ave Apt 3a pgh pa 15207
25,Dana Oree 8-23-18,7/18,McKeesport. PA 1600 Sumace St #5 McKeesport P...,N/A
20,Lisa 8/01/2018,June 2017,"West view, 11 richland ave 2nd fl",NaN
4,Ms. Mary Beth,08/2016,,"2100 Duquesne Pl Dr, Duquesne, PA"
1,Nicola Hor'Tonepomper,07/2010,,"Mckeesport, Evans Ave"
8,Quay,Jan 18th 2018,"Jennyland, McKeesport",NaN
50,Reshawn Cross,8.3.18,"Bradock, Pa",NaN
70,Reshawn Cross 9.27.18,NaN,NaN,"Braddock, Pa"
38,Robin Johnson 8-23-18,Wilmerding Pa 15148 314 Commerce st,,CCR\n704 Market St\nMcKeesport Pa 15132


In [73]:
# Filter out ones that are too ambiguous
ppt_ok_df = ppt_df[(ppt_df.Name!="Karen Lindsey 6-6-18") & (ppt_df.Name!="Lora Owens 8-31-18") & (ppt_df.Name!="Lisa Gonzalez")& (ppt_df.Date.fillna('')!= '') & (ppt_df.start_address.fillna('')!= '') & (ppt_df.dest_address.fillna('')!='') & (ppt_df.dest_address.fillna('')!='N/A')].copy().reset_index()
#ppt_ok_df=ppt_ok_df.set_index('Name')
ppt_ok_df[['Name','Date','start_address', 'dest_address']]
#ppt_ok_df

,Name,Date,start_address,dest_address
0,Nicole Sled,2002,highland park,Duqesne
1,Ralph Williams,02/2017,307 North Matilda St. Garfield,"Allentown, upper Southside"
2,Heidi Morris,12/2017,7529 Hamilton ave / Homewood,South 21st south 711/ Southside
3,Linda Warman,2014,Hazelwood 203 Glenn Caladh St. pgh pa 15207,"2601 Duquesne Place Drive, Duquesne PA 15110"
4,Renee,Oct 2017,Clairton,Clairton Senior Bldg
5,Zhane Cash 6-12-2018,May 9th 2017,718 Junilla St Pgh Pa 15219,"602 Abor Lane Verona, Pa 15147"
6,Leslie Brown,01-15-16,Homewood,Hulton Harbor
7,Kelly Davis,2016,East Liberty,Penn Hills
8,Henry Cook,2015,Garfield,Wilkinsburg
9,Laura Cosby,2017,Mt. Washington,Carnegie


In [164]:
len(ppt_ok_df)

59

In [74]:
# Normalize the dates
import dateparser
ppt_ok_df['parse_date'] = ppt_ok_df['Date'].apply(dateparser.parse)

In [75]:
# Fixup addresses
Rayone_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Rayone')].index)[0]
ppt_ok_df.set_value(Rayone_ix,'start_address','17 Hawkins Village, Rankin, PA 15104')

Ronnika_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Ronnika')].index)[0]
ppt_ok_df.set_value(Ronnika_ix,'start_address','Edgewood Ave, Edgewood, PA 15218')

Jamie_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Jamie L. Childs')].index)[0]
ppt_ok_df.set_value(Jamie_ix,'start_address','1415 Fifth Ave, Pittsburgh, PA 15219')


/workspace/rsargent/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until
/workspace/rsargent/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/workspace/rsargent/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


,index,Timestamp_x,Name,Phone Number?,Email Address?,start_address,How much was your rent?,Were utilities included in the rent cost?,Why did you move?,Date,Where was your childcare located?,What means of transportation did you typically use to get to and from your childcare?,"How long was the trip to get to and from your childcare, including walking and wait times for transit?",How much did you pay for transportation to get to and from your childcare?,How frequently did you go to your childcare?,Where was your social/community activities located?,What means of transportation did you typically use to get to and from your social/community activities ?,"How long was the trip to get to and from your social/community activities, including walking and wait times for transit?",How much did you pay for transportation to get to and from your social/community activities?,How frequently did you go to your social/community activities?,Where was your place of worship located?,What means of transportation did you typically use to get to and from your place of worship?,"How long was the trip to get to and from your place of worship, including walking and wait times for transit?",How much did you pay for transportation to get to and from your place of worship?,How frequently did you go to your place of worship?,"Where was your grocery store, farmers market, or food banks/pantry located?","What means of transportation did you typically use to get to and from your grocery store, farmers market, or food banks/pantry?","How long was the trip to get to and from your grocery store, farmers market, or food banks/pantry, including walking and wait times?","How much did you pay for transportation to get to and from your grocery store, farmers market, or food banks/pantry?","How frequently did you go to your grocery store, farmers market, or food banks/pantry?",Where was your doctor's office located?,What means of transportation did you typically use to get to and from your doctor's office?,"How long was the trip to get to and from your doctor's office, including walking and wait times for transit?",How much did you pay for transportation to get to and from your doctor's office?,How frequently did you go to your doctor's office?,Where was your job located?,What means of transportation did you typically use to get to and from your job?,"How long was the trip to get to and from your job, including walking and wait times for transit?",How much did you pay for transportation to get to and from your job?,How frequently did you go to your job?,Age at the time of your move?,Email Address_x,Timestamp_y,How old are you now?,dest_address,How much is your rent?,Are utilities included in the rent cost?,Where is your childcare located?,What means of transportation do you use to get to and from your childcare?,"How long is the trip to get to and from your childcare, including walking and wait times for transit?",How much do you pay for transportation to get to and from your childcare?,How frequently do you go to your childcare?,Where are your social/community activities located?,What means of transportation do you typically use to get to and from your social/community activities?,"How long is the trip to get to and from your social/community activities, including walking and wait times for transit?",How much do you pay for transportation to get to and from your social/community activities?,How frequently do you go to your social/community activities?,Where is your place of worship located?,What means of transportation do you use to get to and from your place of worship?,"How long is the trip to get to and from your place of worship, including walking and wait times for transit?",How much do you pay for transportation to get to and from your place of worship?,How frequently do you go to your place of worship?,"Where is your grocery store, farmers market, or food banks/pantry located?","What means of transportation do you use to get to and from your grocery store, far

In [154]:
Nicole_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Nicole Sled')].index)[0]
ppt_ok_df.set_value(Nicole_ix,'start_address','Highland Park Pittsburgh, PA 15206')
ppt_ok_df.set_value(Nicole_ix,'dest_address','Duquesne Pennsylvania')

# Hulton Harbor, I'm guessing, is actually supposed to be Hulton Arbors in Verona
Leslie_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Leslie Brown')].index)[0]
ppt_ok_df.set_value(Leslie_ix,'start_address','Homewood, Pittsburgh, PA')
ppt_ok_df.set_value(Leslie_ix,'dest_address','815 Arbor Ln, Verona, PA 15147')

Henry_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Henry Cook')].index)[0]
ppt_ok_df.set_value(Henry_ix,'start_address','Garfield, Pittsburgh, PA')

Laura_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Laura Cosby')].index)[0]
ppt_ok_df.set_value(Laura_ix,'start_address','Mt. Washington, Pittsburgh, PA')
ppt_ok_df.set_value(Laura_ix,'dest_address','Carnegie, PA')

# Hulton Harbor, I'm guessing, is actually supposed to be Hulton Arbors in Verona
Paula_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Paula Reed 6-12-18')].index)[0]
ppt_ok_df.set_value(Paula_ix,'dest_address','815 Arbor Ln, Verona, PA 15147')

Yvette_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Yvette Peterson')].index)[0]
ppt_ok_df.set_value(Yvette_ix,'dest_address','Homewood, Pittsburgh, PA')

Celeste_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Celeste Scott 8-1-18')].index)[0]
ppt_ok_df.set_value(Celeste_ix,'start_address','Lawrenceville, Pittsburgh, PA')

Oscar_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Oscar Mitchell 8-24-18')].index)[0]
ppt_ok_df.set_value(Oscar_ix,'start_address','Swissvale, PA')
ppt_ok_df.set_value(Oscar_ix,'dest_address','Homestead, PA')

Mariette_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Mariette Lane 9.21.18')].index)[0]
ppt_ok_df.set_value(Mariette_ix,'dest_address','Homewood, Pittsburgh, PA')

William_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('William Mitchell')].index)[0]
ppt_ok_df.set_value(William_ix,'start_address','850 Main St East Pittsburgh, PA 15112')

Chakia_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('ChaKia Davis')].index)[0]
ppt_ok_df.set_value(Chakia_ix,'start_address','Mt. Washington, Pittsburgh, PA')
ppt_ok_df.set_value(Chakia_ix,'dest_address','North Oakland, Pittsburgh, PA')

Sara_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Sara Fleet')].index)[0]
ppt_ok_df.set_value(Sara_ix,'dest_address','East Hills, Pittsburgh, PA')

Barry_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Barry Angel')].index)[0]
ppt_ok_df.set_value(Barry_ix,'start_address','Northside, Pittsburgh, PA')

Heather_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Heather Cook')].index)[0]
ppt_ok_df.set_value(Heather_ix,'dest_address','Turtle Creek, PA')

Simon_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Simon Wright')].index)[0]
ppt_ok_df.set_value(Simon_ix,'start_address','Homewood, Pittsburgh, PA')
ppt_ok_df.set_value(Simon_ix,'dest_address','Turtle Creek, PA')

Monique_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Monique')].index)[0]
ppt_ok_df.set_value(Monique_ix,'start_address','535 Center St East Pittsburgh, PA 15112')

Anthony_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Anthony Anderson')].index)[0]
ppt_ok_df.set_value(Anthony_ix,'dest_address','Sharpsburg, PA')

Cheryl_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Cheryl Harris')].index)[0]
ppt_ok_df.set_value(Cheryl_ix,'start_address','Morningside, Pittsburgh, PA')

Tameisha_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Tameisha Robinson')].index)[0]
ppt_ok_df.set_value(Tameisha_ix,'start_address','Hazelwood, Pittsburgh, PA')

Cynthia_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Cynthia Walker')].index)[0]
ppt_ok_df.set_value(Cynthia_ix,'start_address','Oakland, Pittsburgh, PA')

Kari_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Kari Lee')].index)[0]
ppt_ok_df.set_value(Kari_ix,'start_address','Oakland, Pittsburgh, PA')
ppt_ok_df.set_value(Kari_ix,'dest_address','Hazelwood, Pittsburgh, PA')

Clarence_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Clarence Lewis')].index)[0]
ppt_ok_df.set_value(Clarence_ix,'start_address','220 Larimer Ave Pittsburgh, PA 15206')

Jason_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Jason White')].index)[0]
ppt_ok_df.set_value(Jason_ix,'start_address','220 Larimer Ave Pittsburgh, PA 15206')

Ralph_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Ralph Williams')].index)[0]
ppt_ok_df.set_value(Ralph_ix,'dest_address','Allentown, Pittsburgh, PA')


/workspace/rsargent/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/workspace/rsargent/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until
/workspace/rsargent/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/workspace/rsargent/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/workspace/rsargent/anaconda2/lib/python2.7/site-packages/ipykernel

,level_0,Age at the time of your move?,Are utilities included in the rent cost?,Date,Did you have to change jobs?,Email Address?,Email Address_x,Email Address_y,How frequently did you go to your childcare?,How frequently did you go to your doctor's office?,"How frequently did you go to your grocery store, farmers market, or food banks/pantry?",How frequently did you go to your job?,How frequently did you go to your place of worship?,How frequently did you go to your social/community activities?,How frequently do you go to your childcare?,How frequently do you go to your doctor's office?,"How frequently do you go to your grocery store, farmers market, or food banks/pantry?",How frequently do you go to your job?,How frequently do you go to your place of worship?,How frequently do you go to your social/community activities?,How has the move affected your health and well-being?,"How long is the trip to get to and from your childcare, including walking and wait times for transit?","How long is the trip to get to and from your doctor's office, including walking and wait times for transit?","How long is the trip to get to and from your grocery store, farmers market, or food banks/pantry, including walking and wait times for transit?","How long is the trip to get to and from your job, including walking and wait times for transit?","How long is the trip to get to and from your place of worship, including walking and wait times for transit?","How long is the trip to get to and from your social/community activities, including walking and wait times for transit?","How long was the trip to get to and from your childcare, including walking and wait times for transit?","How long was the trip to get to and from your doctor's office, including walking and wait times for transit?","How long was the trip to get to and from your grocery store, farmers market, or food banks/pantry, including walking and wait times?","How long was the trip to get to and from your job, including walking and wait times for transit?","How long was the trip to get to and from your place of worship, including walking and wait times for transit?","How long was the trip to get to and from your social/community activities, including walking and wait times for transit?",How much did you pay for transportation to get to and from your childcare?,How much did you pay for transportation to get to and from your doctor's office?,"How much did you pay for transportation to get to and from your grocery store, farmers market, or food banks/pantry?",How much did you pay for transportation to get to and from your job?,How much did you pay for transportation to get to and from your place of worship?,How much did you pay for transportation to get to and from your social/community activities?,How much do you pay for transportation to get to and from your childcare?,How much do you pay for transportation to get to and from your doctor's office?,"How much do you pay for transportation to get to and from your grocery store, farmers market, or food banks/pantry?",How much do you pay for transportation to get to and from your job?,How much do you pay for transportation to get to and from your place of worship?,How much do you pay for transportation to get to and from your social/community activities?,How much is your rent?,How much was your rent?,How old are you now?,If so have you been able to find employment with similar amount of pay?,Name,Phone Number?,Timestamp_x,Timestamp_y,Were utilities included in the rent cost?,What means of transportation did you typically use to get to and from your childcare?,"What means of transportation did you typically use to get to and from your grocery store, farmers market, or food banks/pantry?",What means of transportation did you typically use to get to and from your place of worship?,What means of transportation did you typically use to get to and from your doctor's office?,What means of transportation did you typically use to get to and from your job?

In [155]:
# Normalize the addresses
ppt_ok_df['canonical_start_address'] = ppt_ok_df['start_address'].apply(get_canonical_address_google)
ppt_ok_df['canonical_dest_address'] = ppt_ok_df['dest_address'].apply(get_canonical_address_google)

In [156]:
# Fixup issues with dates
Constancetein_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Constancetein')].index)[0]
ppt_ok_df.set_value(Constancetein_ix,'parse_date','2016-07-15')

Shawna_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Shawna')].index)[0]
ppt_ok_df.set_value(Shawna_ix,'parse_date','2018-01-15')

Cynthia_ix = list(ppt_ok_df[ppt_ok_df.Name.str.contains('Cynthia Walker')].index)[0]
ppt_ok_df.set_value(Cynthia_ix,'parse_date','2016-01-01')


ppt_ok_df[['Name','Date','parse_date','start_address', 'dest_address','canonical_start_address','canonical_dest_address']]

/workspace/rsargent/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until
/workspace/rsargent/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/workspace/rsargent/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


,Name,Date,parse_date,start_address,dest_address,canonical_start_address,canonical_dest_address
0,Nicole Sled,2002,2002-11-30,"Highland Park Pittsburgh, PA 15206",Duquesne Pennsylvania,"Highland Park, Pittsburgh, PA 15206, USA","Duquesne, PA, USA"
1,Ralph Williams,02/2017,2017-02-28,307 North Matilda St. Garfield,"Allentown, Pittsburgh, PA","307 N Mathilda St, Pittsburgh, PA 15224, USA","Allentown, Pittsburgh, PA, USA"
2,Heidi Morris,12/2017,2017-12-30,7529 Hamilton ave / Homewood,South 21st south 711/ Southside,"7529 Hamilton Blvd, Trexlertown, PA 18087, USA","711 S 21st St, Pittsburgh, PA 15203, USA"
3,Linda Warman,2014,2014-11-30,Hazelwood 203 Glenn Caladh St. pgh pa 15207,"2601 Duquesne Place Drive, Duquesne PA 15110","203 Glen Caladh St, Pittsburgh, PA 15207, USA","2601 Duquesne Pl Dr, Duquesne, PA 15110, USA"
4,Renee,Oct 2017,2017-10-30,Clairton,Clairton Senior Bldg,"Clairton, PA, USA","Clairton, PA, USA"
5,Zhane Cash 6-12-2018,May 9th 2017,2017-05-09,718 Junilla St Pgh Pa 15219,"602 Abor Lane Verona, Pa 15147","718 Junilla St, Pittsburgh, PA 15219, USA","602 Arbor Ln, Verona, PA 15147, USA"
6,Leslie Brown,01-15-16,2016-01-15,"Homewood, Pittsburgh, PA","815 Arbor Ln, Verona, PA 15147","Homewood North, Pittsburgh, PA, USA","815 Arbor Ln, Verona, PA 15147, USA"
7,Kelly Davis,2016,2016-11-30,East Liberty,Penn Hills,"East Liberty, Pittsburgh, PA, USA","Penn Hills, PA, USA"
8,Henry Cook,2015,2015-11-30,"Garfield, Pittsburgh, PA",Wilkinsburg,"Garfield, Pittsburgh, PA, USA","Wilkinsburg, PA, USA"
9,Laura Cosby,2017,2017-11-30,"Mt. Washington, Pittsburgh, PA","Carnegie, PA","Mt Washington, Pittsburgh, PA, USA","Carnegie, PA 15106, USA"


In [157]:
# Add in people Anne and Randy know
ar_df = pd.DataFrame({'Name':pd.Series(['Mumina', 'Audrey', 'Christie']), 
                      'start_address':pd.Series(['3156 Griffiths St, Pittsburgh, PA 15213', '106 Oakland Ct, Pittsburgh, PA 15213', '3206 Dawson St, Pittsburgh, PA 15213']),
                      'dest_address':pd.Series(['7056 Kelly St, Pittsburgh, PA 15208', '5228 Second Avenue, Pittsburgh, PA 15207', 'Beltzhoover Pittsburgh, PA']),
                      'Date':pd.Series(['7/1/2016', '5/19/2018', '7/1/2017'])
                     })

# Normalize the dates
ar_df['parse_date'] = ar_df['Date'].apply(dateparser.parse)

# Normalize the addresses
ar_df['canonical_start_address'] = ar_df['start_address'].apply(get_canonical_address_google)

ar_df['canonical_dest_address'] = ar_df['dest_address'].apply(get_canonical_address_google)

ar_df[['Name','Date','parse_date','start_address', 'dest_address','canonical_start_address','canonical_dest_address']]

,Name,Date,parse_date,start_address,dest_address,canonical_start_address,canonical_dest_address
0,Mumina,7/1/2016,2016-07-01,"3156 Griffiths St, Pittsburgh, PA 15213","7056 Kelly St, Pittsburgh, PA 15208","3156 Griffiths St, Pittsburgh, PA 15213, USA","7056 Kelly St, Pittsburgh, PA 15208, USA"
1,Audrey,5/19/2018,2018-05-19,"106 Oakland Ct, Pittsburgh, PA 15213","5228 Second Avenue, Pittsburgh, PA 15207","106 Oakland Ct, Pittsburgh, PA 15213, USA","5228 Second Avenue, Pittsburgh, PA 15207, USA"
2,Christie,7/1/2017,2017-07-01,"3206 Dawson St, Pittsburgh, PA 15213","Beltzhoover Pittsburgh, PA","3206 Dawson St, Pittsburgh, PA 15213, USA","Beltzhoover, Pittsburgh, PA, USA"


In [158]:
ppt_ok_df_hold = ppt_ok_df

In [159]:
ppt_ok_df = pd.concat([ppt_ok_df_hold, ar_df],axis=0).reset_index()
ppt_ok_df[['Name','Date','parse_date','start_address', 'dest_address','canonical_start_address','canonical_dest_address']]

/workspace/rsargent/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


ValueError: cannot insert level_0, already exists

In [160]:
# Adapted from http://localhost:8815/notebooks/uwsgi/dotmaptiles/server/gradeck/Process%20Gradeck%20Moves-FromGabriel-AW1.ipynb
import array, calendar, csv, json, math, random, re, time, datetime
import numpy as np

def LngLatToPixelXY(lnglat):
    (lng, lat) = lnglat
    x = (lng + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

def CubicBezier(t, p0, p1, p2, p3):
    return ((1-t)**3 * p0) + (3 * (1-t)**2 * t * p1) + (3 * (1-t)*t**2 * p2) + t**3*p3

def FormatEpochMonthStart(datestr):
    month_start_date = datetime.datetime.strptime(datestr[0:7], '%Y-%m')
    return calendar.timegm(month_start_date.utctimetuple())

def FormatEpochYearStart(datestr):
    month_start_date = datetime.datetime.strptime(datestr[0:4], '%Y')
    return calendar.timegm(month_start_date.utctimetuple())

def add_one_month(t):
    """Return a `datetime.date` or `datetime.datetime` (as given) that is
    one month later.
    
    Note that the resultant day of the month might change if the following
    month has fewer days:
    
        >>> add_one_month(datetime.date(2010, 1, 31))
        datetime.date(2010, 2, 28)
    """
    one_day = datetime.timedelta(days=1)
    one_month_later = t + one_day
    while one_month_later.month == t.month:  # advance to start of next month
        one_month_later += one_day
    target_month = one_month_later.month
    while one_month_later.day < t.day:  # advance to appropriate day
        one_month_later += one_day
        if one_month_later.month != target_month:  # gone too far
            one_month_later -= one_day
            break
    return one_month_later

def add_one_year(t):
    """Return a `datetime.date` or `datetime.datetime` (as given) that is
    one year later.
    """
    one_year = datetime.timedelta(days=365)
    one_year_later = t + one_year
    return one_year_later

def FormatEpochNextMonth(datestr):
    month_start_date = datetime.datetime.strptime(datestr[0:7], '%Y-%m')
    next_month_start_date = add_one_month(month_start_date)
    return calendar.timegm(next_month_start_date.utctimetuple())

def FormatEpochNextYear(datestr):
    month_start_date = datetime.datetime.strptime(datestr[0:4], '%Y')
    next_month_start_date = add_one_year(month_start_date)
    return calendar.timegm(next_month_start_date.utctimetuple())

def FormatEpoch(datestr):
    return calendar.timegm(time.strptime(datestr, '%Y%m%d'))

def PackColor(color):
    return color[0] + color[1] * 256.0 + color[2] * 256.0 * 256.0;

In [29]:
FormatEpochMonthStart(str(ppt_ok_df.iloc[0]['parse_date']))

1036108800

In [30]:
FormatEpochYearStart(str(ppt_ok_df.iloc[0]['parse_date']))

1009843200

In [31]:
FormatEpochNextYear(str(ppt_ok_df.iloc[0]['parse_date']))

1041379200

In [32]:
coords = get_canonical_coords_google(ppt_ok_df.iloc[0]['canonical_start_address'])
print "%f, %f" %(coords.y,coords.x)

42.181692, -87.800344


In [33]:
# Adapted from http://localhost:8815/notebooks/uwsgi/dotmaptiles/server/gradeck/Process%20Gradeck%20Moves-FromGabriel-AW1.ipynb
# Monthly
points = []
local_points = []
local_lat_min = 40.370045
local_lat_max = 40.537980
local_lon_min = -80.067465
local_lon_max = -79.779478

monthcount = {}

try:
    for i in range(0,len(ppt_ok_df)):
        move_date = str(ppt_ok_df.iloc[i]['parse_date'])
        move_epoch = FormatEpochMonthStart(move_date)
        # Always set arrival to be start of month after start of move
        arrival_epoch = FormatEpochNextMonth(move_date)
        o_coords = get_canonical_coords_google(ppt_ok_df.iloc[i]['canonical_start_address'])
        d_coords = get_canonical_coords_google(ppt_ok_df.iloc[i]['canonical_dest_address'])
        o_lat = float(o_coords.y)
        o_lng = float(o_coords.x)
        d_lat = float(d_coords.y)
        d_lng = float(d_coords.x)
        points += LngLatToPixelXY((o_lng, o_lat))
        points += LngLatToPixelXY(((o_lng+d_lng)*0.5, (o_lat+d_lat)*0.5))
        points += LngLatToPixelXY((d_lng, d_lat))
        points.append(move_epoch)
        points.append(arrival_epoch)

        # Check if in range to add for local
        if(d_lat < local_lat_max and d_lat > local_lat_min and
           d_lng < local_lon_max and d_lng > local_lon_min and
           o_lat < local_lat_max and o_lat > local_lat_min and
           o_lng < local_lon_max and o_lng > local_lon_min):
            local_points += LngLatToPixelXY((o_lng, o_lat))
            local_points += LngLatToPixelXY(((o_lng+d_lng)*0.5, (o_lat+d_lat)*0.5))
            local_points += LngLatToPixelXY((d_lng, d_lat))
            local_points.append(move_epoch)
            local_points.append(arrival_epoch)
            #print "Local %f %f" % (d_lat,d_lng)

        monthstr = move_date[0:7]
        if monthstr in monthcount.keys():
            monthcount[monthstr]=monthcount[monthstr]+1
        else:
            monthcount[monthstr]=1
        #rgb = (random.randint(0, 255),random.randint(0, 255),random.randint(0, 255))
        #points.append(PackColor(rgb))
        print "[%d]: [%d](%f,%f)->(%f,%f)[%d], len %d"%(i,move_epoch,
                                                              o_lng,o_lat,
                                                              d_lng,d_lat,
                                                              arrival_epoch,len(points))
    
except Exception as e:
    print "Error processing row %d: %s %s %s" % (i, ppt_ok_df.iloc[i]['Name'], ppt_ok_df.iloc[i]['parse_date'], e)
    
array.array('f', points).tofile(open('ppt/ppt-movers-1.bin', 'wb'))   
array.array('f', local_points).tofile(open('ppt/ppt-local-movers-1.bin', 'wb'))   

[0]: [1036108800](-87.800344,42.181692)->(-79.990586,40.437452)[1038700800], len 8
[1]: [1485907200](-79.946168,40.467264)->(-75.471410,40.602294)[1488326400], len 16
[2]: [1512086400](-75.603952,40.548837)->(-79.976398,40.425578)[1514764800], len 24
[3]: [1414800000](-79.941972,40.409662)->(-79.864998,40.379856)[1417392000], len 32
[4]: [1506816000](-79.881715,40.292292)->(-79.881715,40.292292)[1509494400], len 40
[5]: [1493596800](-79.972783,40.450103)->(-79.804855,40.513453)[1496275200], len 48
[6]: [1451606400](-97.220929,37.710687)->(-117.209054,32.725181)[1454284800], len 56
[7]: [1477958400](-79.926266,40.464348)->(-79.839216,40.501179)[1480550400], len 64
[8]: [1446336000](-93.974089,36.448405)->(-79.881994,40.441736)[1448928000], len 72
[9]: [1509494400](-71.303272,44.270585)->(-98.603673,35.103670)[1512086400], len 80
[10]: [341884800](-79.840881,40.398958)->(-117.209054,32.725181)[344476800], len 88
[11]: [1383264000](-79.856337,40.458147)->(-79.805372,40.513680)[1385856000]

In [161]:
# Adapted from http://localhost:8815/notebooks/uwsgi/dotmaptiles/server/gradeck/Process%20Gradeck%20Moves-FromGabriel-AW1.ipynb
# Do 1 year at a time
points = []
local_points = []
local_lat_min = 40.370045
local_lat_max = 40.537980
local_lon_min = -80.067465
local_lon_max = -79.779478

monthcount = {}

try:
    for i in range(0,len(ppt_ok_df)):
        move_date = str(ppt_ok_df.iloc[i]['parse_date'])
        move_epoch = FormatEpochYearStart(move_date)
        # Always set arrival to be start of year after start of move
        arrival_epoch = FormatEpochNextYear(move_date)
        o_coords = get_canonical_coords_google(ppt_ok_df.iloc[i]['canonical_start_address'])
        d_coords = get_canonical_coords_google(ppt_ok_df.iloc[i]['canonical_dest_address'])
        o_lat = float(o_coords.y)
        o_lng = float(o_coords.x)
        d_lat = float(d_coords.y)
        d_lng = float(d_coords.x)
        points += LngLatToPixelXY((o_lng, o_lat))
        points += LngLatToPixelXY(((o_lng+d_lng)*0.5, (o_lat+d_lat)*0.5))
        points += LngLatToPixelXY((d_lng, d_lat))
        points.append(move_epoch)
        points.append(arrival_epoch)

        # Check if in range to add for local
        if(d_lat < local_lat_max and d_lat > local_lat_min and
           d_lng < local_lon_max and d_lng > local_lon_min and
           o_lat < local_lat_max and o_lat > local_lat_min and
           o_lng < local_lon_max and o_lng > local_lon_min):
            local_points += LngLatToPixelXY((o_lng, o_lat))
            local_points += LngLatToPixelXY(((o_lng+d_lng)*0.5, (o_lat+d_lat)*0.5))
            local_points += LngLatToPixelXY((d_lng, d_lat))
            local_points.append(move_epoch)
            local_points.append(arrival_epoch)
            #print "Local %f %f" % (d_lat,d_lng)

        monthstr = move_date[0:7]
        if monthstr in monthcount.keys():
            monthcount[monthstr]=monthcount[monthstr]+1
        else:
            monthcount[monthstr]=1
        #rgb = (random.randint(0, 255),random.randint(0, 255),random.randint(0, 255))
        #points.append(PackColor(rgb))
        
        print "[%d]: [%d](%f,%f)->(%f,%f)[%d], len %d"%(i,move_epoch,
                                                              o_lng,o_lat,
                                                              d_lng,d_lat,
                                                              arrival_epoch,len(points))
    
except Exception as e:
    print "Error processing row %d: %s %s %s" % (i, ppt_ok_df.iloc[i]['Name'], ppt_ok_df.iloc[i]['parse_date'], e)
    
array.array('f', points).tofile(open('ppt/ppt-movers-3.bin', 'wb'))   
array.array('f', local_points).tofile(open('ppt/ppt-local-movers-3.bin', 'wb'))   

[0]: [1009843200](-79.916472,40.479870)->(-79.859771,40.381458)[1041379200], len 8
[1]: [1483228800](-79.946168,40.467264)->(-79.993398,40.422303)[1514764800], len 16
[2]: [1483228800](-75.603952,40.548837)->(-79.976398,40.425578)[1514764800], len 24
[3]: [1388534400](-79.941972,40.409662)->(-79.864998,40.379856)[1420070400], len 32
[4]: [1483228800](-79.881715,40.292292)->(-79.881715,40.292292)[1514764800], len 40
[5]: [1483228800](-79.972783,40.450103)->(-79.804855,40.513453)[1514764800], len 48
[6]: [1451606400](-79.889882,40.458630)->(-79.805675,40.513345)[1483142400], len 56
[7]: [1451606400](-79.926266,40.464348)->(-79.839216,40.501179)[1483142400], len 64
[8]: [1420070400](-79.937458,40.467850)->(-79.881994,40.441736)[1451606400], len 72
[9]: [1483228800](-80.011570,40.427669)->(-80.083388,40.408680)[1514764800], len 80
[10]: [315532800](-79.840881,40.398958)->(-79.805675,40.513345)[347068800], len 88
[11]: [1356998400](-79.856337,40.458147)->(-79.805372,40.513680)[1388534400], 

In [162]:
# Adapted from http://localhost:8815/notebooks/uwsgi/dotmaptiles/server/Allegheny%20County%20Property%20Assessments-AW1.ipynb
def color_from_floats(r,g,b):
    return r + g * 256.0 + b * 256.0 * 256.0

def pack_color(color):
    return color['r'] + color['g'] * 256.0 + color['b'] * 256.0 * 256.0;

def parse_color(color):
    color = color.strip()
    c = color
    try:
        if c[0] == '#':
            c = c[1:]
        if len(c) == 3:
            return pack_color({'r': 17 * int(c[0:1], 16),
                               'g': 17 * int(c[1:2], 16),
                               'b': 17 * int(c[2:3], 16)})
        if len(c) == 6:
            return pack_color({'r': int(c[0:2], 16),
                               'g': int(c[2:4], 16),
                               'b': int(c[4:6], 16)})
    except:
        pass
    raise InvalidUsage('Cannot parse color <code><b>%s</b></code> from spreadsheet.<br><br>Color must be in standard web form, <code><b>#RRGGBB</b></code>, where RR, GG, and BB are each two-digit hexadecimal numbers between 00 and FF.<br><br>See <a href="https://www.w3schools.com/colors/colors_picker.asp">HTML Color Picker</a>' % color)

def parse_colors(colors):
    packed = [parse_color(color) for color in colors]
    return numpy.array(packed, dtype = numpy.float32)

In [163]:
# Adapted from http://localhost:8815/notebooks/uwsgi/dotmaptiles/server/Allegheny%20County%20Property%20Assessments-AW1.ipynb
   
# Colors
src_color = '#ffffff'
dest_color = '#ff0000'

points = []
try:
    for i in range(0,len(ppt_ok_df)):
        move_date = str(ppt_ok_df.iloc[i]['parse_date'])
        move_epoch = FormatEpochYearStart(move_date)
        # Always set arrival to be start of year after start of move
        arrival_epoch = FormatEpochNextYear(move_date)
        o_coords = get_canonical_coords_google(ppt_ok_df.iloc[i]['canonical_start_address'])
        d_coords = get_canonical_coords_google(ppt_ok_df.iloc[i]['canonical_dest_address'])
        o_lat = float(o_coords.y)
        o_lng = float(o_coords.x)
        d_lat = float(d_coords.y)
        d_lng = float(d_coords.x)
        
        # Add origin dot
        points += LngLatToPixelXY((o_lng, o_lat))
        points.append(parse_color(src_color))
        points.append(move_epoch)
        points.append(float(1e38))

        # Add destination dot
        points += LngLatToPixelXY((d_lng, d_lat))
        points.append(parse_color(dest_color))
        points.append(arrival_epoch)
        points.append(float(1e38))

except Exception as e:
    print "Error processing row %d: %s %s %s" % (i, ppt_ok_df.iloc[i]['Name'], ppt_ok_df.iloc[i]['parse_date'], e)
    
array.array('f', points).tofile(open('ppt/ppt-static-epoch-3.bin', 'wb'))